# Conversational agent (LangChain 1.2.x / 2025–2026 style)

This notebook is a **drop-in modernization** of a 2023 LangChain tutorial that used now-obsolete patterns like:
- legacy `langchain.chat_models.ChatOpenAI`
- OpenAI *function-calling* utilities like `format_tool_to_openai_function`
- ad-hoc scratchpad plumbing (`OpenAIFunctionsAgentOutputParser`, `format_to_openai_functions`, etc.)
- `ConversationBufferMemory` (superseded for agents by **LangGraph checkpointers**)

In LangChain **1.2.x**, the recommended approach is to use:
- `langchain_openai.ChatOpenAI` (provider package)
- tools via the `@tool` decorator
- graph-based agents via `langchain.agents.create_agent` (built on LangGraph)
- short-term memory using a LangGraph **checkpointer** (thread-level persistence)

## 0) Environment + safety checks

This notebook is designed to run in a **Python 3.13** environment with:
- `langchain==1.2.0`
- `langchain-core==1.2.4`
- `langchain-openai==1.1.6`
- `langgraph==1.0.5`
- `openai==2.14.0`

If you *don't* have `OPENAI_API_KEY` set, the notebook will still run end-to-end using a **fake chat model** for deterministic, offline smoke tests.

In [1]:
import os
import sys
import importlib
from importlib.metadata import version as pkg_version

REQUIRED = {
    "langchain": "1.2.0",
    "langchain-core": "1.2.4",
    "langchain-openai": "1.1.6",
    "langgraph": "1.0.5",
    "openai": "2.14.0",
}

print("Python:", sys.version)

missing = []
mismatched = []

for pkg, expected in REQUIRED.items():
    try:
        got = pkg_version(pkg)
        if got != expected:
            mismatched.append((pkg, expected, got))
    except Exception:
        missing.append(pkg)

if missing:
    raise RuntimeError(f"Missing required packages: {missing}")

if mismatched:
    msg = "\n".join([f"- {pkg}: expected {exp}, got {got}" for pkg, exp, got in mismatched])
    raise RuntimeError("Package version mismatch:\n" + msg)

print("✅ Package versions match the expected environment.")
print("OPENAI_API_KEY set:", bool(os.environ.get("OPENAI_API_KEY")))

Python: 3.13.11 | packaged by conda-forge | (main, Dec  6 2025, 11:37:04) [Clang 19.1.7 ]
✅ Package versions match the expected environment.
OPENAI_API_KEY set: True


## 1) Define tools (2025/2026 tool-calling style)

A *tool* is a typed callable the model can invoke.  
In 2023, many tutorials converted tools into OpenAI function schemas manually.

In LangChain 1.2.x you do **not** do that yourself:
- define a tool with `@tool` (schema inferred from type hints), and
- pass tools to `create_agent(...)`

The agent runtime handles the tool calling protocol.

In [2]:
from typing import Literal
import datetime
import requests
from pydantic import BaseModel, Field

from langchain.tools import tool

class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> str:
    """Fetch the current temperature (°C) for a given (lat, lon) using Open-Meteo."""
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "timezone": "UTC",
    }

    r = requests.get(base_url, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()

    # Find the temperature closest to 'now' in UTC
    now = datetime.datetime.now(datetime.timezone.utc)
    times = [
        datetime.datetime.fromisoformat(t).replace(tzinfo=datetime.timezone.utc)
        for t in data["hourly"]["time"]
    ]
    temps = data["hourly"]["temperature_2m"]

    idx = min(range(len(times)), key=lambda i: abs(times[i] - now))
    return f"The current temperature is {temps[idx]}°C (UTC time: {times[idx].isoformat()})."

In [3]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia and return a short summary for the top result."""
    try:
        title = wikipedia.search(query, results=1)
        if not title:
            return "No Wikipedia results found."
        page = wikipedia.page(title[0], auto_suggest=False)
        summary = wikipedia.summary(page.title, sentences=3, auto_suggest=False)
        return f"Title: {page.title}\nSummary: {summary}"
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Your query is ambiguous. Options include: {', '.join(e.options[:8])}..."
    except Exception as e:
        return f"Wikipedia error: {type(e).__name__}: {e}"

In [4]:
@tool
def create_your_own(query: str) -> str:
    """Example custom tool: reverse the user's string."""
    return query[::-1]

In [5]:
tools = [get_current_temperature, search_wikipedia, create_your_own]
[t.name for t in tools]

['get_current_temperature', 'search_wikipedia', 'create_your_own']

## 2) Create a modern agent (LangGraph-powered)

In LangChain 1.2.x, the recommended agent entry point is:

```python
from langchain.agents import create_agent
```

This returns a **graph runnable** (LangGraph under the hood).  
For multi-turn conversation, use a **checkpointer** and provide a `thread_id`.

In [6]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# ---- model selection ----
# If no OPENAI_API_KEY is present, we fall back to a deterministic fake chat model
# so the notebook still runs end-to-end.
def get_model():
    api_key = os.environ.get("OPENAI_API_KEY")
    if api_key:
        return ChatOpenAI(model="gpt-5-mini", temperature=0)
    # Offline: deterministic fake model for smoke testing
    from langchain_core.language_models.fake_chat_models import FakeListChatModel
    from langchain_core.messages import AIMessage
    return FakeListChatModel(responses=[
        AIMessage(content="(offline demo) I can't call external tools without a real model, but the agent wiring works.")
    ])

model = get_model()

memory = InMemorySaver()

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="You are helpful but a little sassy. Be concise.",
    checkpointer=memory,
)

# Each conversation thread is identified by thread_id.
CONFIG = {"configurable": {"thread_id": "demo-thread-1"}}

## 3) Invoke the agent

The agent expects a state update containing messages.

We pass:
```python
{"messages": [{"role": "user", "content": "..."}]}
```

and (when using memory) a config containing:
```python
{"configurable": {"thread_id": "..."}}
```

In [7]:
def ask(user_text: str):
    result = agent.invoke(
        {"messages": [{"role": "user", "content": user_text}]},
        CONFIG,
    )
    # Agent returns a state dict with 'messages'
    return result["messages"][-1].content

ask("What is LangChain?")

'Short answer: LangChain is an open-source framework for building applications that use large language models (LLMs). It wraps LLMs with tools, memory, and orchestration so you can build more capable, reliable, and data-aware apps than "single-prompt → response."\n\nWhat it does (high level)\n- Connects LLMs to data, APIs, and tools (databases, Google Drive, search, calculators, web scraping, etc.).\n- Provides building blocks to compose multi-step "chains" of prompts and logic.\n- Adds memory and state so conversations/apps can be context-aware.\n- Offers "agents" that let models decide which tools to call and when.\n\nCore concepts (brief)\n- LLM wrappers: standard interface to different models (OpenAI, local models, Hugging Face, etc.).\n- Prompts & prompt templates: reusable, parameterized prompts.\n- Chains: sequences of operations (prompts, transformations, tool calls).\n- Agents & Tools: let the model choose and execute external actions (e.g., run a search, call an API).\n- Memo

In [8]:
ask("My name is Bob.")

'Nice to meet you, Bob — I’ll remember that for this chat. What can I help you with?'

In [9]:
ask("What's my name?")

"Your name's Bob. Got it — I’ll remember that for this chat."

## 4) Tool-using prompts (weather + Wikipedia)

If you have a valid `OPENAI_API_KEY`, the agent can decide to call tools.

Example prompts:
- "What's the weather at lat 37.7749, lon -122.4194?"
- "Wikipedia: what is LangChain?"

If you're running offline (no API key), you'll still get a clean response without errors.

In [10]:
ask("What's the weather at latitude 37.7749 and longitude -122.4194?")

'The current temperature at latitude 37.7749, longitude -122.4194 is 12.0°C.'

In [11]:
ask("Wikipedia: what is LangChain?")

'From Wikipedia: LangChain is a software framework that helps integrate large language models (LLMs) into applications. It supports use cases like document analysis, summarization, chatbots, and code analysis. It was launched in October 2022 by Harrison Chase while at Robust Intelligence.'

## 5) A tiny chat loop (no extra UI dependencies)

The original 2023 notebook used `panel` for a GUI.  
This environment does **not** require any GUI libs: run a minimal terminal-style loop instead.

Stop with an empty input.

In [12]:
def chat():
    print("Type a message (empty to stop).")
    while True:
        user = input("you> ").strip()
        if not user:
            break
        print("bot>", ask(user))

# Uncomment to use interactively in a local Jupyter session:
chat()

Type a message (empty to stop).


you>  what is the temperature in New York City


bot> The current temperature in New York City (lat 40.7128, lon -74.0060) is 1.3°C.


you>  Wikipedia: what is the hibert hotel? And why don't real numbers fit in that hotel?


bot> You mean Hilbert’s Hotel — a famous thought experiment (from Wikipedia and elsewhere) by David Hilbert illustrating strange properties of infinite sets.

What Hilbert’s Hotel is
- Imagine a hotel with rooms numbered 1, 2, 3, … (one room for every natural number). It’s full: every room has a guest.
- Despite being “full,” it can still accommodate more guests by reassigning rooms. For example:
  - To fit one more guest, move the guest in room n to room n+1; room 1 becomes free.
  - To fit countably infinitely many new guests, move the guest in room n to room 2n; all odd-numbered rooms become free for the newcomers.
- This shows counterintuitive behavior of countably infinite sets: they can be put in one-to-one correspondence with proper subsets of themselves.

Why the real numbers don’t fit
- The hotel’s rooms correspond to the natural numbers, so it can only host sets that are countable (i.e., can be listed as a sequence r1, r2, r3, …).
- The real numbers are uncountable: there is 

...

you>  Show why most reals are uncomputable or transcendental.


bot> Short answer: because the sets of algorithms (programs) and of algebraic numbers are both countable, while the real numbers are uncountable — so almost every real number (in the sense of cardinality and Lebesgue measure) is neither produced by any algorithm nor a root of any integer-coefficient polynomial.

Proof sketches (concise)

1) Most reals are uncomputable
- A computable real is one for which some finite program outputs arbitrarily good rational approximations.
- Programs are finite strings over a finite alphabet, so the set of all programs is countable.
- Each program can define at most one real number, so the set of computable reals is countable.
- The real numbers are uncountable (Cantor), so there are uncountably many reals that aren’t computable. Hence “most” reals are uncomputable.

2) Most reals are transcendental
- An algebraic number is a root of some nonzero polynomial with integer coefficients.
- For each fixed degree n, the set of integer-coefficient polynomials

you>  wikipedia: Walk through Cantor’s diagonal proof step-by-step.


bot> Sure — let’s walk through Cantor’s diagonal argument step by step. I’ll use decimal expansions in the interval (0,1) (the same idea works in binary and for other sets). I’ll point out the little technical wrinkle about repeating 9s and how to avoid it.

Goal: show that the real numbers in (0,1) cannot be listed as r1, r2, r3, … (i.e., they are uncountable).

1. Assume the opposite (for contradiction).
   - Suppose every real in (0,1) appears somewhere in an infinite list:
     r1, r2, r3, …

2. Write each rn in decimal form.
   - rn = 0.dn1 dn2 dn3 … where dnk is the k-th decimal digit of rn.
   - To avoid ambiguity (e.g., 0.4999… = 0.5000…), choose for each real its decimal expansion that does NOT end in an infinite string of 9s. This is always possible.

3. Construct a new number x by changing the diagonal digits.
   - Define x = 0.x1 x2 x3 … where each xn is chosen so that xn ≠ dnn.
   - For instance, set xn = 5 if dnn ≠ 5, and xn = 4 if dnn = 5. (Any rule that guarantees xn di

you>  
